# Use LMM simulation for Normal Response instead of Element Wise Simulation + Additional Hessian Term

Instead of simulating the random vector element by element and then doing this 100,000 times, here we check the robustness of the estimation procedure by using a more standard way to simulate from the LMM model. 

$n = 100,000 \text{ replicates}$

$Y_i \sim N(X \beta, \sigma^2 = 0.1), i \in [1, n]$ 

$\beta = \begin{pmatrix} 1 \\ 1\\ 1 \end{pmatrix}$

$n_i = 20, i \in [1, n]$

$V_1 = 1_n 1_n^t, V_2 = I_n$

$\Gamma = 0.1 * V_1 + 0.1 * V_2$

In [1]:
using GLMCopula, Random, LinearAlgebra, GLM

Random.seed!(123)

n = 100_000  # number of observations
n_i = 20
ns = [n_i for i in 1:n]
p = 3   # number of mean parameters
m = 1   # number of variance component in addition to normal noise (estimated separately as precision τ)

d = Normal()
link = IdentityLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, n)
# true parameter values
βtruth = ones(p)
σ2truth = [0.1; 0.1]
σ02truth = 0.1
for i in 1:n
    ni = ns[i]
    V1 = ones(ni, ni)
    V2 = Matrix(I, ni, ni)
    Ω = σ2truth[1] * V1 + σ2truth[1] * V2 + σ02truth * I
    Ωchol = cholesky(Symmetric(Ω))
    # simulate design matrix
    X = [ones(ni) randn(ni, p-1)]
    # generate mvn response 
    y = X * βtruth + Ωchol.L * randn(ni)
    # add to data
    gcs[i] = GLMCopulaVCObs(y, X, [V1, V2], d, link)
end

gcm = GLMCopulaVCModel(gcs);

In [2]:
@info "Initial point:"
initialize_model!(gcm)
@show gcm.β
fill!(gcm.Σ, 1)
update_Σ!(gcm)
@show inv.(gcm.τ)
@show gcm.Σ
loglikelihood!(gcm, true, true)

┌ Info: Initial point:
└ @ Main In[2]:1


gcm.β = [0.9989814332272863, 1.0000312910023397, 1.00059386282601]
inv.(gcm.τ) = [0.27643235256683074]
gcm.Σ = [0.43424661863044045, 1.5189655117548047e-16]


-1.5343093590991343e6

In [3]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, hessian_approximation = "exact"))

gcm.β = [0.9989814332272863, 1.0000312910023397, 1.00059386282601]
gcm.Σ = [0.43424535496726735, 1.1208475900674373e-16]
gcm.β = [0.9989814332272863, 1.0000312910023397, 1.00059386282601]
gcm.Σ = [0.4342441993934382, 8.270747671416851e-17]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............

In [4]:
@show gcm.β
@show inv.(gcm.τ)
@show gcm.Σ
@show loglikelihood!(gcm, true, true);

gcm.β = [0.998958293850988, 1.000222637774681, 1.0006237508208964]
inv.(gcm.τ) = [0.2764324109945519]
gcm.Σ = [0.43423802638105036, 5.086890653848633e-155]
loglikelihood!(gcm, true, true) = -1.5343092231882308e6


### Now with a different V1, V2

In [6]:
d = Normal()
link = IdentityLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, n)
# true parameter values
βtruth = ones(p)
σ2truth = [0.1; 0.1]
σ02truth = 0.1
for i in 1:n
    ni = ns[i]
    # set up covariance matrix
    V1 = convert(Matrix, Symmetric([Float64(i * (ni - j + 1)) for i in 1:ni, j in 1:ni])) # a pd matrix
    V1 ./= norm(V1) / sqrt(ni) # scale to have Frobenius norm sqrt(n)
    prob = fill(1/ni, ni)
    V2 = ni .* (Diagonal(prob) - prob * transpose(prob))
    V2 ./= norm(V2) / sqrt(ni) # scale to have Frobenious norm sqrt(n)
    Ω = σ2truth[1] * V1 + σ2truth[2] * V2 + σ02truth * I
    Ωchol = cholesky(Symmetric(Ω))
    # simulate design matrix
    X = [ones(ni) randn(ni, p-1)]
    # generate mvn response 
    y = X * βtruth + Ωchol.L * randn(ni)
    # add to data
    gcs[i] = GLMCopulaVCObs(y, X, [V1, V2], d, link)
end

gcm = GLMCopulaVCModel(gcs);

In [7]:
@info "Initial point:"
initialize_model!(gcm)
@show gcm.β
fill!(gcm.Σ, 1)
update_Σ!(gcm)
@show inv.(gcm.τ)
@show gcm.Σ
loglikelihood!(gcm, true, true)

┌ Info: Initial point:
└ @ Main In[7]:1


gcm.β = [0.9985697077170472, 1.0004534779716074, 0.9996962195892055]
inv.(gcm.τ) = [0.21610959142947544]
gcm.Σ = [0.9283841980590499, 2.4978283552677564e-8]


-1.3571057603921946e6

In [8]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, hessian_approximation = "exact"))

gcm.β = [0.9985697077170472, 1.0004534779716074, 0.9996962195892055]
gcm.Σ = [0.9283823527519064, 2.393217562243978e-8]
gcm.β = [0.9985697077170472, 1.0004534779716074, 0.9996962195892055]
gcm.Σ = [0.9283805598840092, 2.2929875377080854e-8]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with onl

gcm.Σ = [0.9283339725493271, 2.7954854718696707e-10]
  34  1.3571057e+06 0.00e+00 5.32e-06  -8.6 6.02e-12    -  1.00e+00 1.00e+00f  1
gcm.β = [0.9987161333491841, 1.0004352678774506, 0.9996276912424132]
gcm.Σ = [0.9283337682409396, 2.4587231425204817e-10]
  35  1.3571057e+06 0.00e+00 4.88e-06  -8.6 5.52e-12    -  1.00e+00 1.00e+00f  1
gcm.β = [0.9987161333445232, 1.0004352678756534, 0.9996276912474721]
gcm.Σ = [0.9283335809368941, 2.1625292662093253e-10]
  36  1.3571057e+06 0.00e+00 4.47e-06  -8.6 5.06e-12    -  1.00e+00 1.00e+00f  1
gcm.β = [0.9987161333402503, 1.0004352678740056, 0.99962769125211]
gcm.Σ = [0.9283334092230593, 1.9020167425029136e-10]
  37  1.3571057e+06 0.00e+00 4.10e-06  -8.6 4.64e-12    -  1.00e+00 1.00e+00f  1
gcm.β = [0.998716133336333, 1.000435267872495, 0.999627691256362]
gcm.Σ = [0.928333251802832, 1.6728871935089892e-10]
  38  1.3571057e+06 0.00e+00 3.76e-06  -8.6 4.25e-12    -  1.00e+00 1.00e+00f  1
gcm.β = [0.9987161333327417, 1.0004352678711101, 0.999627691

In [9]:
@show gcm.β
@show inv.(gcm.τ)
@show gcm.Σ
@show loglikelihood!(gcm, true, true);

gcm.β = [0.9987161332962026, 1.0004352678570156, 0.9996276912999226]
inv.(gcm.τ) = [0.2161097962752476]
gcm.Σ = [0.9283316393474583, 3.26487438712995e-12]
loglikelihood!(gcm, true, true) = -1.357105670829792e6
